STEP #0: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

STEP #1: Initial Exploration

In [ ]:
# Load data
df = pd.read_csv('../data/raw/data.csv', encoding='ISO-8859-1')

# First look - shape and first few rows
print('Shape:', df.shape)
print('\nFirst 10 rows:')
display(df.head(10))

# Column info
print('\nColumn info:')
display(df.info())

# Basic Statistics - describe
print('\nBasic Statics')
display(df.describe())
# Missing values
display(df.isnull().sum())

# Unique values per column
print('Unique values per column:')
for col in df:
    display(f'\n{col}: {df[col].nunique()}')


STEP #2: Data Quality Check

In [ ]:
# Check for problems

# 1. Missing CustomerIDs
print(f"Missing customer IDs: {df['CustomerID'].isnull().sum()}")
print(f"Percentage: {df['CustomerID'].isnull().sum() / len(df) * 100:.2f}%")

# 2. Negative quantities (returns/cancellations)
print(f"Negative quantities: {(df['Quantity'] < 0).sum()}")
print(f"Negative prices: {(df['UnitPrice'] < 0).sum()}")

# 3. Cancelled orders (check InvoiceNo starting with 'C')
if df['InvoiceNo'].dtype == 'object':
    cancelled = df['InvoiceNo'].astype(str).str.startswith('C').sum()
    print(f"Cancelled invoices: {cancelled}")

# 4. Countries
print(f"\nCountries represented: {df['Country'].nunique()}")
print(f"\nTop Countries by transaction: {df['Country'].value_counts().head(11)}")

# 5. Date range
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")

# 6. Check for duplicates
print(f"Duplicate rows: {df.duplicated().sum()}")


STEP #3: Create SQLite Database

In [ ]:
# Create database connection
conn = sqlite3.connect('../data/ecommerce.db')

# Load raw data into SQL
df.to_sql('transaction_raw', conn, if_exists = 'replace', index = False)

# Test it works
test_query = " SELECT COUNT(*) as total_rows FROM transaction_raw"
result = pd.read_sql(test_query, conn)
print(f"Rows in database: {result['total_rows'][0]}")

STEP #4: Exploratory SQL queries

In [ ]:
# Query 1: Total transactions and revenue
query1 = """
    SELECT
        COUNT(DISTINCT InvoiceNo) as total_invoices,
        COUNT(*) as total_line_items,
        SUM(Quantity * UnitPrice) as total_revenue
    FROM transaction_raw
    WHERE Quantity > 0 AND UnitPrice > 0
"""
print("Overall statistics:")
display(pd.read_sql(query1, conn))

# Query 2: Revenue by country
query2 = """
SELECT
    Country,
    COUNT(DISTINCT InvoiceNo) as num_orders,
    SUM(Quantity * UnitPrice) as revenue
FROM transaction_raw
WHERE Quantity > 0 AND UnitPrice > 0
GROUP by Country
ORDER by revenue DESC
LIMIT 10
"""
print("\nTop 10 countries by revenue:")
display(pd.read_sql(query2, conn))


# Query 3: Top products
query3 = """
SELECT
    Description,
    SUM(Quantity) as units_sold,
    SUM(Quantity * UnitPrice) as revenue
FROM transaction_raw
WHERE Quantity > 0 AND UnitPrice > 0
GROUP by Description
ORDER by revenue DESC
LIMIT 10
"""
print("\nTop 10 products:")
display(pd.read_sql(query3, conn))

# Query 4: Monthly revenue trend
query4 = """
SELECT
    strftime('%Y-%m', InvoiceDate) as month,
    SUM(Quantity * UnitPrice) as revenue
FROM transaction_raw
WHERE Quantity > 0 AND UnitPrice > 0
GROUP by month
ORDER by month
"""
print("\nMonthly revenue:")
display(pd.read_sql(query4, conn))

# Query 5: Customer count
query5 = """
SELECT
    COUNT(DISTINCT CustomerID) as unique_customers
FROM transaction_raw
WHERE CustomerID IS NOT NULL
"""
print("\nUnique customers:")
display(pd.read_sql(query5, conn))
